In [1]:
from featureextraction import generate_embeddings
from eval_embedding_rs import predict_labels_based_on_distance
import h5py
import matplotlib.pyplot as plt
from models import get_baseline_convolutional_encoder, build_siamese_net
import numpy as np
import os
import random
from scipy.signal import decimate
from sklearn.manifold import TSNE
import tensorflow as tf
import yaml

In [7]:
# define species set
dataset_folds = [line.rstrip().split(',') for line in open('cv_folds.txt')]
species_set = dataset_folds[0]

# load parameters from config_file
config_file = 'siamrs13_4c_ew32_25.yaml'
models_path = 'models'
model_file = os.path.join(models_path, config_file)
with open(model_file) as cfile:
    config_params = yaml.safe_load(cfile)
eeg_epoch_width_in_s = config_params['epoch_width']
num_classes = config_params['num_classes']
decimate_factor = 4

In [3]:
# read EEG epochs - randomly sample num_samples from each class of each species
num_samples = 128
epochs_path = 'data/epochs_{}c'.format(num_classes)
file_template = '{}_BL5_ew{}.h5'
sw_s_epochs = []
ss_s_epochs = []
tw_s_epochs = []
ts_s_epochs = []
for species in species_set:
    print('Working on', species)
    file_path = os.path.join(epochs_path, file_template.format(species, eeg_epoch_width_in_s))
    with h5py.File(file_path, 'r') as file:
        if 'Sham' in species:
            sw_s_epochs.extend(random.sample(list(file['eeg']['wake']), num_samples))
            ss_s_epochs.extend(random.sample(list(file['eeg']['sleep']), num_samples))
        elif 'TBI' in species:
            tw_s_epochs.extend(random.sample(list(file['eeg']['wake']), num_samples))
            ts_s_epochs.extend(random.sample(list(file['eeg']['sleep']), num_samples))

Working on Sham104
Working on Sham105
Working on Sham107
Working on Sham108
Working on TBI103
Working on TBI104
Working on TBI106
Working on Sham102
Working on Sham103
Working on TBI101
Working on TBI102


In [4]:
# generate embeddings for EEG epochs
sw_epochs = generate_embeddings(decimate(sw_s_epochs, decimate_factor), model_file)
ss_epochs = generate_embeddings(decimate(ss_s_epochs, decimate_factor), model_file)
tw_epochs = generate_embeddings(decimate(tw_s_epochs, decimate_factor), model_file)
ts_epochs = generate_embeddings(decimate(ts_s_epochs, decimate_factor), model_file)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [5]:
# reduce dimensionality using t-SNE
tsne = TSNE(perplexity=30, early_exaggeration=12.0, learning_rate=600.0, method='exact', init='pca', verbose=1,
            n_iter=40000)
sw_epochs = tsne.fit_transform(sw_epochs)
ss_epochs = tsne.fit_transform(ss_epochs)
tw_epochs = tsne.fit_transform(tw_epochs)
ts_epochs = tsne.fit_transform(ts_epochs)

[t-SNE] Computing pairwise distances...
[t-SNE] Computed conditional probabilities for sample 768 / 768
[t-SNE] Mean sigma: 3.819419
[t-SNE] KL divergence after 250 iterations with early exaggeration: 63.766579
[t-SNE] KL divergence after 7050 iterations: 0.452443
[t-SNE] Computing pairwise distances...
[t-SNE] Computed conditional probabilities for sample 768 / 768
[t-SNE] Mean sigma: 3.676067
[t-SNE] KL divergence after 250 iterations with early exaggeration: 65.021561
[t-SNE] KL divergence after 7000 iterations: 0.449196
[t-SNE] Computing pairwise distances...
[t-SNE] Computed conditional probabilities for sample 640 / 640
[t-SNE] Mean sigma: 4.502927
[t-SNE] KL divergence after 250 iterations with early exaggeration: 63.169674
[t-SNE] KL divergence after 6550 iterations: 0.402334
[t-SNE] Computing pairwise distances...
[t-SNE] Computed conditional probabilities for sample 640 / 640
[t-SNE] Mean sigma: 4.822864
[t-SNE] KL divergence after 250 iterations with early exaggeration: 64.7

In [ ]:
# plot embeddings
plt.figure(figsize=(24.0, 24.0))
fig, ax = plt.subplots()
ax.scatter(sw_epochs[:, 0], sw_epochs[:, 1], label='sw')
ax.scatter(ss_epochs[:, 0], ss_epochs[:, 1], label='ss')
ax.scatter(tw_epochs[:, 0], tw_epochs[:, 1], label='tw')
ax.scatter(ts_epochs[:, 0], ts_epochs[:, 1], label='ts')
ax.legend()
ax.grid(True)
plt.savefig('{}_all.png'.format(config_file))
plt.show()

In [ ]:
predicted_labels = predict_labels_based_on_distance()